In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
test_dir = '../input/animal-faces/afhq/val'
train_dir = '../input/animal-faces/afhq/train'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen = ImageDataGenerator(rescale = 1./255, validation_split= 0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (100, 100),
    batch_size = 128,
    class_mode = "categorical",
    color_mode = "rgb",
    subset = "training"
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (100, 100),
    batch_size = 128,
    class_mode = "categorical",
    color_mode = "rgb",
    subset = "validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (100, 100),
    batch_size = 128,
    class_mode = "categorical",
    color_mode = "rgb"
)

In [ ]:
checkpoint = ModelCheckpoint('model_dog_cat_wild.hdf5',monitor='val_accuracy', verbose= 1, save_best_only=True)
stop = EarlyStopping(monitor='val_accuracy',patience=2)

In [ ]:
model_base = Sequential()

model_base.add(Conv2D(32, (3,3), activation='relu', input_shape=(100,100,3)))
model_base.add(MaxPooling2D((2,2)))

model_base.add(Conv2D(64, (3,3), activation='relu'))
model_base.add(MaxPooling2D((2,2)))

model_base.add(Conv2D(128, (3,3), activation='relu'))
model_base.add(MaxPooling2D((2,2)))

model_base.add(Flatten())
model_base.add(Dropout(0.2))
model_base.add(Dense(64,activation='relu'))
model_base.add(Dropout(0.3))
model_base.add(Dense(128,activation='relu'))
model_base.add(Dropout(0.2))
model_base.add(Dense(256,activation='relu'))
model_base.add(Dropout(0.3))
model_base.add(Dense(3,activation='softmax'))

model_base.summary()

In [ ]:
model_base.compile(optimizer= "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

history_optimizer = model_base.fit(
    train_generator,
    epochs = 20,
    validation_data = validation_generator,
    callbacks = [checkpoint],
    verbose = 1
)

In [ ]:
model_base.evaluate(test_generator)

In [ ]:
import matplotlib.pyplot as plt
def visualizacion_resultados(history):
  epochs = [i for i in range(20)]
  fig, ax = plt.subplots(1,2)
  train_acc = history.history["accuracy"]
  train_loss = history.history["loss"]
  val_acc = history.history["val_accuracy"]
  val_loss = history.history["val_loss"]
  fig.set_size_inches(16, 9)

  ax[0].plot(epochs, train_acc, "go-", label =" Entrenamiento accuracy")
  ax[0].plot(epochs, val_acc, "ro-", label= "Validacion accuracy")
  ax[0].set_title("Entrenamiento & validación accuracy")
  ax[0].legend()
  ax[0].set_xlabel("Epochs")
  ax[0].set_ylabel("Accuracy")

  ax[1].plot(epochs, train_loss, "go-", label =" Entrenamiento Loss")
  ax[1].plot(epochs, val_loss, "ro-", label= "Validacion Loss")
  ax[1].set_title("Entrenamiento & validación Loss")
  ax[1].legend()
  ax[1].set_xlabel("Epochs")
  ax[1].set_ylabel("Loss")

  plt.show()

In [ ]:
visualizacion_resultados(history_optimizer)

In [ ]:
model2=model_base
model2.load_weights('./model_dog_cat_wild.hdf5')

In [ ]:
model2.evaluate(test_generator)

In [ ]:
config_cnn = model_base.get_config()
print(config_cnn)

In [ ]:
model_base.save("saved_model_complete2/mymodel_good")

In [ ]:
import matplotlib.image as mpimg
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 5, figsize = (10, 10))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img[:,:,0])
    ax.axis("off")
  plt.tight_layout()
  plt.show()
    
sample_training_images, _= next(train_generator)
plotImages(sample_training_images[:5])

In [ ]:
from keras.preprocessing import image
import numpy as np
test_image = image.load_img('../input/animal-faces/afhq/train/dog/flickr_dog_000152.jpg', target_size = (100, 100,3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model_base.predict(test_image)
train_generator.class_indices

if result[0][0] == 1:
    prediction = 'cat'
elif result[0][1] == 1:
    prediction = 'dog'
elif result[0][2] == 1:
    prediction = 'wild'

imagen = image.load_img('../input/animal-faces/afhq/train/dog/flickr_dog_000152.jpg', target_size = (200, 200,3)) 
print(prediction)
print(result)
plt.imshow(imagen)
plt.show()

In [ ]:
model_base.save('model.h5')

In [ ]:
!mkdir carpeta_salida

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format keras model.h5 carpeta_salida